In [2]:
import sys
sys.path.insert(0, '/tf/utils/')


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import numpy as np
import os
# Definir o nível de log do TensorFlow para ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Activation, Dropout, Conv1D, Conv1DTranspose, MaxPooling1D, concatenate, Concatenate, LSTM, Dense, SimpleRNN, BatchNormalization, Dropout, BatchNormalization, Add, Flatten, UpSampling1D, UpSampling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LayerNormalization, Dropout, MultiHeadAttention, Conv2D, Conv2DTranspose, MaxPooling2D, Permute, Reshape

# import sounddevice as sd
from scipy.signal import butter, filtfilt
from scipy.signal import resample
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.io import wavfile
from glob2 import glob
from tqdm import tqdm
from pesq import pesq
import matplotlib.pyplot as plt


from utils import load_wav, generate_white_gaussian_noise, add_white_gaussian_noise, undersample_signal_with_antialiasing, somar_sinais, calculate_snr
from sound import Sound

In [4]:
base_shape_size = 2000

In [5]:
sound_base = Sound('./Base/', './ESC-50-master/audio/', base_shape_size)

Loading noise files: 100%|█████████████████| 2000/2000 [00:08<00:00, 236.19it/s]


In [16]:
def calcular_componentes_fourier(sinal):
    if len(sinal.shape) == 3 and sinal.shape[2] == 1:
        sinal = sinal.reshape(sinal.shape[0], sinal.shape[1])

    componentes = np.fft.fft(sinal)
    # Obter a magnitude máxima dos componentes
    # max_magnitude = np.max(np.abs(componentes), axis=1).reshape(-1, 1)
    # Normalizar os componentes para o intervalo [-1, 1]
    max_magnitude = 1
    # componentes_normalizados = componentes / max_magnitude
    res = np.stack((np.real(componentes), np.imag(componentes)), axis=-1)

    return np.array(res), max_magnitude

def reconstruir_sinal(componentes, max_magnitude):
    comp = componentes[:, :, 0] + 1j * componentes[:, :, 1]
#     print(comp.shape)
    sinal_reconstruido = np.fft.ifft(comp) * max_magnitude
    return np.real(sinal_reconstruido)

In [24]:
class DataGenerator:
    def __init__(self, sound_files, noise_files):
        self.sound_files = sound_files
        self.noise_files = noise_files
        self.MIN_NOISE_LEVEL = 5
        self.MAX_NOISE_LEVEL = 30
        
        self.model_batch = load_model('chech.h5')

    def generate_sample_completo(self, batch_size=32):
        while True:
            # Carrega um lote de sons
            sound_batch_choices = np.random.choice(self.sound_files.shape[0], size=batch_size, replace=False)
            sound_batch = self.sound_files[sound_batch_choices]
            
            # Carrega um lote de ruídos
            noise_batch_choices = np.random.choice(self.noise_files.shape[0], size=batch_size, replace=False)
            noise_batch = self.noise_files[noise_batch_choices]
            
            x1_train = []
            x2_train = []
            y_train = []
            
            # Adiciona ruído a cada som e calcula a nota PESQ
            for sound, noise in zip(sound_batch, noise_batch):
                sr = np.random.randint(-5, 60, size=(1,)[0])
                noisy_sound = somar_sinais(sound, noise, sr)
                
                noisy_sound = add_white_gaussian_noise(noisy_sound, np.random.randint(5, 30, size=(1,)[0]))
                noisy_sound = np.clip(noisy_sound, -1, 1)
                
                comp, _ = calcular_componentes_fourier(noisy_sound.reshape(-1, base_shape_size))
                original_sound, _ = calcular_componentes_fourier(sound.reshape(-1, base_shape_size))
                
#                 filtered = self.model_batch.predict(
#                     np.concatenate((original_sound.reshape(base_shape_size, 2), filtered.reshape(base_shape_size, 2)), axis=-1)
#                     comp.reshape(-1, base_shape_size, 2), verbose=0)
#                 filtered_sound = reconstruir_sinal(filtered, 1)
    
                # Calcula a nota PESQ
                pesq_score = 1.0
#                 try:
#                     pesq_score = pesq(8000, sound, filtered_sound.reshape(-1), 'nb')
#                 except:
#                     continue
#                 valor_min = -0.6
#                 valor_max = 4.6
#                 pesq_score = (pesq_score - valor_min) / (valor_max - valor_min)
#                 pesq_score = min(pesq_score + 0.2, 1.0)
                
                # Formata dados para treinamento
                original_sound = original_sound.reshape((base_shape_size, 2))
                d_sound = comp.reshape((base_shape_size, 2))
                
#                 merged_array = np.concatenate((original_sound, d_sound), axis=1)
                
                # Adiciona o exemplo ao lote de treinamento
                x1_train.append(d_sound)
                x2_train.append(original_sound)
                y_train.append(pesq_score)
            
            yield [np.array(x1_train), np.array(x2_train)], np.array(y_train)

In [25]:
# Carregar o modelo treinado
modelo_pesq = load_model('chech.h5')

# Definir seus pesos como não treináveis
for layer in modelo_pesq.layers:
    layer.trainable = False

In [26]:
model_pre = load_model('modelo-pre-fourier-09-06-2023.h5')

2023-08-14 04:08:15.641160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-14 04:08:15.642572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-14 04:08:15.643572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [27]:
input_novo = Input(shape=(base_shape_size, 2))
input_concat = Input(shape=(base_shape_size, 2))

output_novo = model_pre(input_novo)
output_concat = concatenate([input_concat, output_novo], axis=-1)

completo = modelo_pesq(output_concat)

modelo_final = Model(inputs=[input_novo, input_concat], outputs=completo)

2023-08-14 04:08:39.744041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-14 04:08:39.745347: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-14 04:08:39.746235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [28]:
modelo_final.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 2000, 2)]    0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 2000, 2)]    0           []                               
                                                                                                  
 model_8 (Functional)           (None, 2000, 2)      4831490     ['input_5[0][0]']                
                                                                                                  
 concatenate_2 (Concatenate)    (None, 2000, 4)      0           ['input_6[0][0]',                
                                                                  'model_8[0][0]']          

In [29]:
# Crie uma instância do DataGenerator
data_generator_train = DataGenerator(sound_base.train_X, sound_base.noise_sounds)
data_generator_val = DataGenerator(sound_base.val_X, sound_base.noise_sounds)

In [30]:
validation_batch = next(data_generator_val.generate_sample_completo(batch_size=32))
x_val, y_val = validation_batch

In [31]:
x_val[0].shape

(32, 2000, 2)

In [32]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

# Definir taxa de aprendizado e fator de decaimento
learning_rate = 1e-5
decay_factor = 0.1

# Definir taxa de aprendizado para as camadas adicionais
additional_layers_lr = 0.001

# Compilar o modelo final com os ajustes
optimizer = Adam(learning_rate=learning_rate)
modelo_final.compile(optimizer='sgd', loss='mse')

# # Ajustar a taxa de aprendizado das camadas adicionais
# for layer in modelo_final.layers:
#     if layer.trainable:
#         layer.learning_rate = additional_layers_lr

# Definir tamanho do lote e número de passos por época
batch_size = 64
steps_per_epoch = len(sound_base.train_X) // batch_size

# Definir decaimento da taxa de aprendizado
lr_decay = ReduceLROnPlateau(factor=decay_factor)

print('Starting training')

for epoch in range(5):
    print(f"Epoch {epoch + 1}")
    
    # Gera um novo lote de validação para cada época
    validation_batch = next(data_generator_val.generate_sample_completo(batch_size=batch_size))
    x_val, y_val = validation_batch
    
    modelo_final.fit(data_generator_train.generate_sample_completo(batch_size=batch_size),
                     steps_per_epoch=steps_per_epoch,
                     epochs=1,
                     validation_data=(x_val, y_val),
#                      callbacks=[lr_decay]
                    )

    inferencia_modelo_novo = Model(inputs=input_novo, outputs=output_novo)
    inferencia_modelo_novo.compile(optimizer=optimizer, loss='mse')
    inferencia_modelo_novo.save('epoch-{}-refinado_na_pesq.h5'.format(epoch+1))

Starting training
Epoch 1


2023-08-14 04:08:47.896989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-14 04:08:48.078303: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-14 04:08:48.079379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

509/509 [==============================] - ETA: 0s - loss: 0.1095

2023-08-14 04:10:53.717603: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-08-14 04:10:53.718884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-08-14 04:10:53.719884: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

509/509 [==============================] - 127s 241ms/step - loss: 0.1095 - val_loss: 0.0950
Epoch 2


...

509/509 [==============================] - 124s 243ms/step - loss: 0.0898 - val_loss: 0.0726
Epoch 4


2023-08-14 04:15:01.755538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


509/509 [==============================] - 123s 242ms/step - loss: 0.0843 - val_loss: 0.0752
Epoch 5


2023-08-14 04:17:05.167118: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


509/509 [==============================] - 124s 243ms/step - loss: 0.0803 - val_loss: 0.0835


In [33]:
inferencia_modelo_novo = Model(inputs=input_novo, outputs=output_novo)
inferencia_modelo_novo.compile(optimizer=optimizer, loss='mse')
inferencia_modelo_novo.save('refinado_na_pesq_4p0e-5.h5')